# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from dateutil.relativedelta import relativedelta
import math
import pandas_profiling as pp
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from os import cpu_count
from sklearn.svm import SVR


n_jobs = max(cpu_count()-1,1)

In [93]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("wikipedia_train3.csv")
test = pd.read_csv("wikipedia_test3.csv")

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [94]:
train.head(1), train.tail(1)

(                                                Page        date  Visits
 0  Special:Search_zh.wikipedia.org_all-access_spider  2016-01-01  2660.0,
                                                       Page        date  \
 2882859  Wikipedia:Portada_es.wikipedia.org_all-access_...  2016-08-31   
 
           Visits  
 2882859  57752.0  )

In [95]:
test.head(1), test.tail(1)

(                                                Page        date  Visits
 0  Special:Search_zh.wikipedia.org_all-access_spider  2016-09-10   640.0,
                                                      Page        date   Visits
 732529  Wikipedia:Portada_es.wikipedia.org_all-access_...  2016-11-10  56413.0)

In [96]:
train["date"] = pd.to_datetime(train["date"])

In [97]:
test["date"] = pd.to_datetime(test["date"])

In [98]:
#  from 10 to 10 date validation with no gap 

In [99]:
start_val_day = '2016-06-10'

In [102]:
train_validation = train[(train['date']>=start_val_day) & (train['date']<='2016-08-10')].copy()

In [103]:
train_validation.head(1),train_validation.tail(1)

(                                                      Page       date  Visits
 1902215  Special:Search_zh.wikipedia.org_all-access_spider 2016-06-10  2259.0,
                                                       Page       date   Visits
 2634744  Wikipedia:Portada_es.wikipedia.org_all-access_... 2016-08-10  57590.0)

In [104]:
start_val_day = pd.to_datetime(start_val_day)
start_val_day

Timestamp('2016-06-10 00:00:00')

In [106]:
last_train_date = start_val_day - relativedelta(days=10)

In [107]:
train_new = train.loc[train['date'] <= last_train_date].copy() 

In [108]:
train_new.tail()

,Page,date,Visits
1795875,The_Revenant_–_Der_Rückkehrer_de.wikipedia.org...,2016-05-31,3094.0
1795876,Especial:Entrar_es.wikipedia.org_all-access_sp...,2016-05-31,4151.0
1795877,Especial:Libro_es.wikipedia.org_all-access_spider,2016-05-31,1488.0
1795878,Special:Search_es.wikipedia.org_all-access_spider,2016-05-31,15007.0
1795879,Wikipedia:Portada_es.wikipedia.org_all-access_...,2016-05-31,60736.0


In [109]:
lastday = max(train_new['date'])

In [110]:
last_15_days = train_new.loc[train_new['date'] > last_train_date - relativedelta(days=15)]

In [111]:
last_15_days.head(1),last_15_days.tail(1)

(                                                      Page       date   Visits
 1618655  Special:Search_zh.wikipedia.org_all-access_spider 2016-05-17  12192.0,
                                                       Page       date   Visits
 1795879  Wikipedia:Portada_es.wikipedia.org_all-access_... 2016-05-31  60736.0)

In [112]:
median_baseline = train_new.loc[train_new["date"] > last_train_date - relativedelta(days=15)].groupby(['Page'],sort = False).median().reset_index()
median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [113]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [114]:
pandas_smape(test.merge(median_baseline, on="Page", how='left')) #the score test set.

50.35649862558084

In [115]:
pandas_smape(train_validation.merge(median_baseline, on="Page", how='left'))

46.803231996423584

In [ ]:
### Your code here

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [145]:
train['Month']     = train["date"].dt.month
train['Day']       = train["date"].dt.day
train['DayOfWeek'] = train["date"].dt.dayofweek

In [147]:
test['Month']     = test["date"].dt.month
test['Day']       = test["date"].dt.day
test['DayOfWeek'] = test["date"].dt.dayofweek

In [148]:
### Your code here
def create_validation(df, val_start):
    return df.loc[(df['date'] >= pd.to_datetime(val_start) - relativedelta(months=3)) & \
                  (df['date'] <  pd.to_datetime(val_start) - relativedelta(days=10))].index, \
           df.loc[(df['date'] >= pd.to_datetime(val_start) + relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(val_start) + relativedelta(days=62))].index

In [149]:
val_dates = ['2016-04-10', '2016-05-10', '2016-06-10']

In [150]:
### Your validation iterator
CVIterator = []

for i in val_dates:
    trainIndices, valIndices = create_validation(train, i)
    print("trainIndices",trainIndices)
    print("valIndices",valIndices)
    print(i)
    CVIterator.append( (trainIndices, valIndices) )

trainIndices Int64Index([ 106335,  106336,  106337,  106338,  106339,  106340,  106341,
             106342,  106343,  106344,
            ...
            1063340, 1063341, 1063342, 1063343, 1063344, 1063345, 1063346,
            1063347, 1063348, 1063349],
           dtype='int64', length=957015)
valIndices Int64Index([1181500, 1181501, 1181502, 1181503, 1181504, 1181505, 1181506,
            1181507, 1181508, 1181509,
            ...
            1914020, 1914021, 1914022, 1914023, 1914024, 1914025, 1914026,
            1914027, 1914028, 1914029],
           dtype='int64', length=732530)
2016-04-10
trainIndices Int64Index([ 472600,  472601,  472602,  472603,  472604,  472605,  472606,
             472607,  472608,  472609,
            ...
            1417790, 1417791, 1417792, 1417793, 1417794, 1417795, 1417796,
            1417797, 1417798, 1417799],
           dtype='int64', length=945200)
valIndices Int64Index([1535950, 1535951, 1535952, 1535953, 1535954, 1535955, 1535956,
        

In [126]:
train

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0
5,Cristiano_Ronaldo_fr.wikipedia.org_desktop_all...,2016-01-01,1762.0
6,Facebook_fr.wikipedia.org_desktop_all-agents,2016-01-01,2261.0
7,Liste_d'attentats_meurtriers_fr.wikipedia.org_...,2016-01-01,379.0
8,Liste_des_automobiles_Ferrari_fr.wikipedia.org...,2016-01-01,96.0
9,Prince_(musicien)_fr.wikipedia.org_desktop_all...,2016-01-01,300.0


In [127]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [128]:
estimator = LinearRegression()

In [130]:
params = {}

In [151]:
gs = GridSearchCV(
    estimator=estimator,  # base model
    param_grid=params,  # params grid to search within
    cv=CVIterator,  # cross-validation strategy
    error_score=1,  # warnings only
    scoring='neg_mean_squared_error',  # f1-score
    # thread count, the higher count - the faster
    n_jobs=n_jobs,
    verbose=2,  # messages about performed actions
)

In [140]:
X = train.drop(['Visits','date'], axis = 1)

In [141]:
y = train['Visits']

In [152]:
X = train.drop(['Visits','date','Page'], axis = 1)
y = train['Visits']

X_test = test.drop(['Visits','date','Page'], axis = 1)
y_test = test['Visits']

In [156]:
gs.fit(X,y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.2s finished


GridSearchCV(cv=[(Int64Index([ 106335,  106336,  106337,  106338,  106339,  106340,  106341,
             106342,  106343,  106344,
            ...
            1063340, 1063341, 1063342, 1063343, 1063344, 1063345, 1063346,
            1063347, 1063348, 1063349],
           dtype='int64', length=957015), Int64Ind...2634740, 2634741,
            2634742, 2634743, 2634744],
           dtype='int64', length=732530))],
       error_score=1,
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=3, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=2)

In [157]:
val_pred = gs.predict(y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[  640.  5609.  3121. ...  1561.  4624. 56413.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
Visits.values

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [3]:
train = pd.read_csv('train_french.csv')
test = pd.read_csv('test_french.csv')

1. Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

In [4]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [5]:
test.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,6029,5,OUI,500,Famille,agent; employe,Maison,1,7,C,...,0,164,41,0,0,1,NON,30,1,107.94
1,62128,2,NON,0,Famille,enseignant; formateur; chercheur,Appartement,1,1,D,...,1,85,100,0,0,1,NON,30,2,80.41
2,81065,4,NON,1000,Indivividu,homme ou femme au foyer,Sans information,2,99,~,...,0,94,100,0,0,1,NON,30,2,142.12
3,5023,3,NON,500,Indivividu,agent; employe,Maison,2,38,C,...,0,101,100,0,0,5,NON,30,2,76.55
4,62178,2,NON,500,Famille,enseignant; formateur; chercheur,Appartement,1,1,~,...,1,125,100,1,0,1,NON,0,2,150.45


In [6]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [7]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [8]:
train.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
code_postal,200000,NaN,NaN,NaN,53290,27916.9,1001,31526,57463,75119,97801
nb_pieces,200000,NaN,NaN,NaN,2.91366,1.31391,1,2,3,4,5
veranda,200000,2,NON,194793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
biens_nomades,200000,NaN,NaN,NaN,585.49,328.811,0,500,500,1000,2000
sit_fam,200000,2,Famille,102443,NaN,NaN,NaN,NaN,NaN,NaN,NaN
profession,200000,18,enseignant; formateur; chercheur,72666,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nature_lri,200000,6,Appartement,110302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
societe,200000,NaN,NaN,NaN,1.37827,0.484955,1,1,1,2,2
domaine_activite,200000,NaN,NaN,NaN,19.7315,25.1489,0,2,9,34,99
niv_urbanisation,200000,8,A,42970,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
categorical = [
    'veranda', 
    'var9', 
    'sit_fam', 
    'profession', 
    'niv_urbanisation', 
    'nature_lri'
]

binary_cols = ([
    # ...
    'formule',
    'objets_precieux',
    'pres_contrat_auto',
    'societe',
    'var3',
    'var6',
    'var7',
    'var11',
])

In [10]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

__encode_categorical(df_list=[train, test], cat_cols=categorical)       


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 29 columns):
code_postal          200000 non-null int64
nb_pieces            200000 non-null int64
veranda              200000 non-null int64
biens_nomades        200000 non-null int64
sit_fam              200000 non-null int64
profession           200000 non-null int64
nature_lri           200000 non-null int64
societe              200000 non-null int64
domaine_activite     200000 non-null int64
niv_urbanisation     200000 non-null int64
age_societaire       200000 non-null int64
objets_precieux      200000 non-null int64
pres_contrat_auto    200000 non-null int64
formule              200000 non-null int64
sit_juridique        200000 non-null int64
surf_habitat         200000 non-null int64
patrimoine_mob       200000 non-null int64
var1                 200000 non-null int64
var2                 200000 non-null int64
var3                 200000 non-null int64
var4                 200000 n

In [12]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,18
Categorical,0
Boolean,11
Date,0
Text (Unique),0
Rejected,0


In [ ]:
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop('prime_tot_ttc', axis=1)
y = train['prime_tot_ttc']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

In [ ]:
model = XGBRegressor(iterations=1000, max_depth=10, learning_rate=0.6, loss_function='MAE')

In [ ]:
np.array(y_valid)

In [ ]:
model.fit(X=X_train, y=y_train, verbose=True)

In [ ]:
val_prediction = model.predict(X_valid)
test_prediction= model.predict(test.drop('prime_tot_ttc', axis=1))

In [ ]:
def mape(y_true, y_pred):
    return np.sum(100 / y_true.shape[0] * np.abs((y_true - y_pred) / y_true))

In [ ]:
mape(np.array(y_valid), val_prediction)

In [ ]:
mape(np.array(test['prime_tot_ttc']), test_prediction)

# Tree

In [4]:
train = pd.read_csv('train_french.csv')
test = pd.read_csv('test_french.csv')

In [5]:
test.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,6029,5,OUI,500,Famille,agent; employe,Maison,1,7,C,...,0,164,41,0,0,1,NON,30,1,107.94
1,62128,2,NON,0,Famille,enseignant; formateur; chercheur,Appartement,1,1,D,...,1,85,100,0,0,1,NON,30,2,80.41
2,81065,4,NON,1000,Indivividu,homme ou femme au foyer,Sans information,2,99,~,...,0,94,100,0,0,1,NON,30,2,142.12
3,5023,3,NON,500,Indivividu,agent; employe,Maison,2,38,C,...,0,101,100,0,0,5,NON,30,2,76.55
4,62178,2,NON,500,Famille,enseignant; formateur; chercheur,Appartement,1,1,~,...,1,125,100,1,0,1,NON,0,2,150.45


In [6]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [7]:
categorical = [
    'veranda', 
    'var9', 
    'sit_fam', 
    'profession', 
    'niv_urbanisation', 
    'nature_lri',
    
]

binary_cols = ([
    # ...
    'formule',
    'objets_precieux',
    'pres_contrat_auto',
    'societe',
    'var3',
    'var6',
    'var7',
    'var11',
])

In [8]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

__encode_categorical(df_list=[train, test], cat_cols=categorical)       


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = train.drop('prime_tot_ttc', axis=1)
y = train['prime_tot_ttc']

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 28 columns):
code_postal          200000 non-null int64
nb_pieces            200000 non-null int64
veranda              200000 non-null int64
biens_nomades        200000 non-null int64
sit_fam              200000 non-null int64
profession           200000 non-null int64
nature_lri           200000 non-null int64
societe              200000 non-null int64
domaine_activite     200000 non-null int64
niv_urbanisation     200000 non-null int64
age_societaire       200000 non-null int64
objets_precieux      200000 non-null int64
pres_contrat_auto    200000 non-null int64
formule              200000 non-null int64
sit_juridique        200000 non-null int64
surf_habitat         200000 non-null int64
patrimoine_mob       200000 non-null int64
var1                 200000 non-null int64
var2                 200000 non-null int64
var3                 200000 non-null int64
var4                 200000 n

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
)

In [50]:
model = DecisionTreeClassifier(max_depth=3, random_state=42,
            splitter='best')


In [51]:
model.fit(
    X=X_train, y=y_train.astype('int')
)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [52]:
val_pred = model.predict(X_valid)


In [53]:
val_pred

array([85, 61, 61, ..., 97, 68, 97])

In [54]:
test_pred = model.predict(test.drop('prime_tot_ttc', axis=1))

In [55]:
test_pred

array([101,  97, 139, ...,  96,  85, 139])

In [60]:
#def mape(y_true, y_pred):
    #return np.sum(100 / y_true.shape[0] * np.abs((y_true - y_pred) / y_true))

In [59]:
#mape(np.array(y_valid), val_pred)

In [58]:
#mape(np.array(test['prime_tot_ttc']), test_pred)

In [47]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [ ]:
#y_v = pd.DataFrame(y_valid)

In [ ]:
#v_p = pd.DataFrame(val_pred)

In [56]:
smape_fast(y_valid, val_pred)

17.57126411279395

In [57]:
smape_fast(test['prime_tot_ttc'], test_pred)

17.70802700350473

In [ ]:
#val_pred

In [ ]:
#t_v = pd.DataFrame(test['prime_tot_ttc'])

In [ ]:
#t_p = pd.DataFrame(test_pred)

In [40]:
#test_pred

# second variant

In [61]:
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X = train.drop('prime_tot_ttc', axis=1)
y = train['prime_tot_ttc']

In [64]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

In [65]:
model = XGBRegressor(iterations=1000, max_depth=10, learning_rate=0.6, loss_function='MAE')

In [66]:
np.array(y_valid)

array([105.36, 181.65, 102.09, ...,  62.5 , 102.16,  44.16])

In [67]:
model.fit(X=X_train, y=y_train, verbose=True)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, iterations=1000, learning_rate=0.6,
       loss_function='MAE', max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [68]:
val_prediction = model.predict(X_valid)
test_prediction= model.predict(test.drop('prime_tot_ttc', axis=1))

In [69]:
def mape(y_true, y_pred):
    return np.sum(100 / y_true.shape[0] * np.abs((y_true - y_pred) / y_true))

In [70]:
mape(np.array(y_valid), val_prediction)

8.116142457173334

In [71]:
mape(np.array(test['prime_tot_ttc']), test_prediction)

8.550706006401377